In [1]:
import pandas as pd

from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('/mnt/ml-team/minerva/toxic/data/train.csv')
data.shape, data.head()

((159571, 8),
                  id                                       comment_text  toxic  \
 0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
 1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
 2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
 3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
 4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   
 
    severe_toxic  obscene  threat  insult  identity_hate  
 0             0        0       0       0              0  
 1             0        0       0       0              0  
 2             0        0       0       0              0  
 3             0        0       0       0              0  
 4             0        0       0       0              0  )

In [5]:
BWCL_VALID_PATH = ''
CVDCNN_VALID_PATH = ''
CLOGREG_VALID_PATH = ''
GDPCNN_VALID_PATH = ''
GLSTM_VALID_PATH = ''
GSCNN_VALID_PATH = ''
TFIDIF_LOGREG_VALID_PATH = ''
WLSTM_VALID_PATH = ''

VALID_LABELS_PATH = ''

BWCL_TEST_PATH = ''
CVDCNN_TEST_PATH = ''
CLOGREG_TEST_PATH = ''
GDPCNN_TEST_PATH = ''
GLSTM_TEST_PATH = ''
GSCNN_TEST_PATH = ''
TFIDIF_LOGREG_TEST_PATH = ''
WLSTM_TEST_PATH = ''

SAMPLE_SUBMISSION_PATH = ''
ENSEMBLE_SUBMISSION_PATH = ''

# Validation Set Level 1 Predictions

In [6]:
bad_word_count_logreg_valid = pd.read_csv(BWCL_VALID_PATH)
char_vdcnn_valid = pd.read_csv(CVDCNN_VALID_PATH)
count_logreg_valid = pd.read_csv(CLOGREG_VALID_PATH)
glove_dpcnn_valid = pd.read_csv(GDPCNN_VALID_PATH)
glove_lstm_valid = pd.read_csv(GLSTM_VALID_PATH)
glove_scnn_valid = pd.read_csv(GSCNN_VALID_PATH)
tfidf_logreg_valid = pd.read_csv(TFIDIF_LOGREG_VALID_PATH)
word_lstm_valid = pd.read_csv(WLSTM_VALID_PATH)

labels_valid = pd.read_csv(VALID_LABELS_PATH)

NameError: name 'pd' is not defined

In [ ]:
X_valid = np.hstack([bad_word_count_logreg_valid, 
                     char_vdcnn_valid, 
                     count_logreg_valid,
                     glove_dpcnn_valid,
                     glove_lstm_valid,
                     glove_scnn_valid,
                     tfidf_logreg_valid,
                     word_lstm_valid])

y_valid_multilabel = labels_valid

# Test Set Level 1 Predictions

In [ ]:
bad_word_count_logreg_test = pd.read_csv(BWCL_TEST_PATH)
char_vdcnn_test = pd.read_csv(CVDCNN_TEST_PATH)
count_logreg_test = pd.read_csv(CLOGREG_TEST_PATH)
glove_dpcnn_test = pd.read_csv(GDPCNN_TEST_PATH)
glove_lstm_test = pd.read_csv(GLSTM_TEST_PATH)
glove_scnn_test = pd.read_csv(GSCNN_TEST_PATH)
tfidf_logreg_test = pd.read_csv(TFIDIF_LOGREG_TEST_PATH)
word_lstm_test = pd.read_csv(WLSTM_TEST_PATH)

In [ ]:
X_test = np.hstack([bad_word_count_logreg_test, 
                    char_vdcnn_test, 
                    count_logreg_test,
                    glove_dpcnn_test,
                    glove_lstm_test,
                    glove_scnn_test,
                    tfidf_logreg_test,
                    word_lstm_test])

# Ensemble Training

In [ ]:
estimators = []
for y_valid_one_label in y_valid:
    estimator = LogisticRegression()
    estimator.fit(X_valid, y_valid_one_label)
    estimators.append(estimator)

# Ensemble Prediction

In [ ]:
for estimator in estimators:
    y_test_pred_one_label = estimator.predict(X_test)
y_test_pred = 
y_test_pred = np.stack(y_test_pred, axis=0)
y_test_pred = y_test_pred[:, :, 1].transpose()

# Submission

In [ ]:
submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[''] = y_test_pred
submission.to_csv(ENSEMBLE_SUBMISSION_PATH)